<h4>Integrantes</h4>
Ruiz Irisbel - 30.864.236

Sosa Andres - 30.096.794

<h2>Análisis y Predicción de Datos Meteorológicos con Métodos de Interpolación</h2>

<h4>Parte 1: Interpolación de Taylor</h4>

<h4>Parte 2: Interpolación de Lagrange
</h4>

<h4>Parte 3: Interpolación de Hermite
</h4>

<h4>Parte 4: Interpolación Polinómica a Trozos
</h4>